In [3]:
import pandas as pd
import numpy as np

#read in data from csv from folder ../data
crime_df = pd.read_csv('../data/cf_apprissAppend_va2_10_17_2022.csv')
medical_df = pd.read_csv('../data/CF_medicalData_v2.csv')
PII_df = pd.read_csv('../data/dev_sample_HeartTable_10_17_2022.csv')

C:\Users\cayde\AppData\Local\Temp\ipykernel_16312\739831198.py:5: DtypeWarning: Columns (26,27) have mixed types. Specify dtype option on import or set low_memory=False.
  crime_df = pd.read_csv('../data/cf_apprissAppend_va2_10_17_2022.csv')
C:\Users\cayde\AppData\Local\Temp\ipykernel_16312\739831198.py:6: DtypeWarning: Columns (16,17,18,19,20,21,22,23,34,36,49,50,51,52,53,54,55,56,57,58,59,60,62,63,64,74,75,76,78,79,80,85,87,88,89,90,91,92,93,97,98,99,108,123,124) have mixed types. Specify dtype option on import or set low_memory=False.
  medical_df = pd.read_csv('../data/CF_medicalData_v2.csv')


In [4]:
# cut down the dataframe medical_df to only the columns we need 'source_id', 'ENDate'
medical_df = medical_df[['source_id', 'ENDate', 'MiddleName']]
# join the dataframes PII_df and medical_df left on source_id
PII_med_df = pd.merge(PII_df, medical_df, on='source_id', how='left')
PII_med_df = PII_med_df[['source_id', 'FIRST_NAME', 'MiddleName', 'LAST_NAME', 'MASTER_DOB', 'ENDate']]



In [11]:
# cut down crime_df to only the columns we need 'source_id', 'SenderId' 'PersonFirstName' 'PersonMiddleName' 'PersonLastName' 'BookingDate'
crime_df = crime_df[['PK','source_id', 'SenderId', 'PersonFirstName', 'PersonMiddleName', 'PersonLastName', 'BookingDate', 'PersonYearOfBirth']]
# rename the column "pk" to booking_event_id
crime_df = crime_df.rename(columns={'PK': 'booking_event_id'})


In [6]:
# join the dataframes PII_med_df and crime_df right on source_id
Crime_PII_df = pd.merge(PII_med_df, crime_df, on='source_id', how='right')

In [7]:
# for the columns PersonFirstName, PersonMiddleName, PersonLastName remove the first and last character
Crime_PII_df['PersonFirstName'] = Crime_PII_df['PersonFirstName'].str[1:-1]
Crime_PII_df['PersonMiddleName'] = Crime_PII_df['PersonMiddleName'].str[1:-1]
Crime_PII_df['PersonLastName'] = Crime_PII_df['PersonLastName'].str[1:-1]
Crime_PII_df['PersonYearOfBirth'] = Crime_PII_df['PersonYearOfBirth'].str[1:-1]
# this will get rid of all of the [] in the columns

In [8]:
#import regular expressions
import re
# # check to see whether each source_id is associated with a unique sender_id
# Crime_PII_df.groupby('source_id').SenderId.nunique().value_counts()

# For all rows where ENDate is not null, check whether BookingDate occurs after ENDate
Crime_PII_df['DiedBeforeBookingCharge'] = pd.to_datetime(Crime_PII_df['BookingDate']) > pd.to_datetime(Crime_PII_df['ENDate'])

def YOB_match(row):
    if type(row['PersonYearOfBirth']) == str:
        pass
    else:
        return False
    # use regular expression to find if 'FIRST_NAME' column in Crime_PII_df contains 'PersonFirstName' column in Crime_PII_df
    if re.search(row['PersonYearOfBirth'], row['MASTER_DOB']):
        return True
    else:
        return False

Crime_PII_df['YOB_match'] = Crime_PII_df.apply(YOB_match, axis=1)

In [9]:
#import regular expressions
import re

# create a function that returns a boolean value for whether the first name in the crime data matches the first name in the PII data
def first_name_match(row):
    if type(row['PersonFirstName']) == str:
        pass
    else:
        return False
    # use regular expression to find if 'FIRST_NAME' column in Crime_PII_df contains 'PersonFirstName' column in Crime_PII_df
    if re.search(row['FIRST_NAME'], row['PersonFirstName']):
        return True
    else:
        return False

# create a function that returns a boolean value for whether the last name in the crime data matches the last name in the PII data
def last_name_match(row):
    if type(row['PersonLastName']) == str:
        pass
    else:
        return False
    # use regular expression to find if 'LAST_NAME' column in Crime_PII_df contains 'PersonLastName' column in Crime_PII_df
    if re.search(row['LAST_NAME'], row['PersonLastName']):
        return True
    else:
        return False

# create a function that returns a boolean value for whether the middle name in the crime data matches the middle name in the PII data
def middle_name_match(row):
    # check if row['PersonMiddleName'] can be searched with re.search without throwing an error
    if type(row['PersonMiddleName']) == str:
        pass
    else:
        return False
    # use regular expression to find if 'PersonMiddleName' column in Crime_PII_df contains 'PersonMiddleName' column in Crime_PII_df
    if re.search(row['PersonMiddleName'], row['PersonMiddleName']):
        return True
    else:
        return False

Crime_PII_df['FN_match'] = Crime_PII_df.apply(first_name_match, axis=1)
Crime_PII_df['LN_match'] = Crime_PII_df.apply(last_name_match, axis=1)
Crime_PII_df['MN_match'] = Crime_PII_df.apply(middle_name_match, axis=1)

In [10]:
# different combinations of first, middle, and last name matches
Crime_PII_df['FN_LN_match'] = Crime_PII_df['FN_match'] & Crime_PII_df['LN_match']
Crime_PII_df['FN_MN_match'] = Crime_PII_df['FN_match'] & Crime_PII_df['MN_match']
Crime_PII_df['LN_MN_match'] = Crime_PII_df['LN_match'] & Crime_PII_df['MN_match']
Crime_PII_df['FN_LN_MN_match'] = Crime_PII_df['FN_match'] & Crime_PII_df['LN_match'] & Crime_PII_df['MN_match']

In [8]:
# write dataframe to csv
Crime_PII_df.to_csv('../data/out/Confidence_match_crimedata.csv')